In [1]:
# ============================================
# Credit Fraud Competition - Full EDA Script
# - Train/Test: unlabeled
# - Val: labeled (EDA/평가 가능, fit은 train만)
# Outputs saved to ./eda_outputs/
# ============================================

import os
import json
import textwrap
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import ks_2samp, wasserstein_distance, ranksums, skew, kurtosis
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest

# -----------------------------
# Global config
# -----------------------------
RANDOM_SEED = 42
OUT_DIR = "eda_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

In [2]:
# =============================
# 0) Utilities
# =============================
def save_df(df: pd.DataFrame, name: str):
    path = os.path.join(OUT_DIR, f"{name}.csv")
    df.to_csv(path, index=False)
    print(f"[SAVE] {path} ({df.shape[0]} rows, {df.shape[1]} cols)")

def save_fig(name: str):
    path = os.path.join(OUT_DIR, f"{name}.png")
    plt.tight_layout()
    plt.savefig(path, dpi=180)
    plt.close()
    print(f"[SAVE] {path}")

def is_finite_df(df: pd.DataFrame) -> pd.DataFrame:
    return np.isfinite(df.to_numpy())

def ensure_numeric(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    # Convert to numeric if needed (coerce errors to NaN)
    out = df.copy()
    for c in cols:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def summarize_basic(df: pd.DataFrame, cols: List[str], prefix: str) -> pd.DataFrame:
    # Per-column summary including tail quantiles and shape metrics
    arr = df[cols]
    out = pd.DataFrame({
        "feature": cols,
        f"{prefix}_count": [arr[c].notna().sum() for c in cols],
        f"{prefix}_nan": [arr[c].isna().sum() for c in cols],
        f"{prefix}_mean": [arr[c].mean() for c in cols],
        f"{prefix}_std": [arr[c].std() for c in cols],
        f"{prefix}_median": [arr[c].median() for c in cols],
        f"{prefix}_min": [arr[c].min() for c in cols],
        f"{prefix}_q01": [arr[c].quantile(0.01) for c in cols],
        f"{prefix}_q05": [arr[c].quantile(0.05) for c in cols],
        f"{prefix}_q25": [arr[c].quantile(0.25) for c in cols],
        f"{prefix}_q75": [arr[c].quantile(0.75) for c in cols],
        f"{prefix}_q95": [arr[c].quantile(0.95) for c in cols],
        f"{prefix}_q99": [arr[c].quantile(0.99) for c in cols],
        f"{prefix}_max": [arr[c].max() for c in cols],
        f"{prefix}_skew": [skew(arr[c].dropna().to_numpy()) if arr[c].dropna().shape[0] > 2 else np.nan for c in cols],
        f"{prefix}_kurtosis": [kurtosis(arr[c].dropna().to_numpy(), fisher=True) if arr[c].dropna().shape[0] > 3 else np.nan for c in cols],
        f"{prefix}_var": [arr[c].var() for c in cols],
    })
    return out

def iqr_bounds(s: pd.Series) -> Tuple[float, float]:
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    return (q1 - 1.5 * iqr, q3 + 1.5 * iqr)

def compute_psi(expected: np.ndarray, actual: np.ndarray, bins: int = 20) -> float:
    """
    PSI(Population Stability Index)
    - expected: reference distribution (e.g., train)
    - actual: target distribution (e.g., test/val)
    """
    expected = expected[np.isfinite(expected)]
    actual = actual[np.isfinite(actual)]
    if expected.size < 10 or actual.size < 10:
        return np.nan

    # Quantile-based bins on expected (robust)
    quantiles = np.linspace(0, 1, bins + 1)
    cuts = np.unique(np.quantile(expected, quantiles))
    if cuts.size < 3:  # too few unique cut points
        return np.nan

    # histogram counts
    e_counts, _ = np.histogram(expected, bins=cuts)
    a_counts, _ = np.histogram(actual, bins=cuts)

    e_pct = e_counts / max(e_counts.sum(), 1)
    a_pct = a_counts / max(a_counts.sum(), 1)

    # add epsilon to avoid div by 0
    eps = 1e-8
    e_pct = np.clip(e_pct, eps, None)
    a_pct = np.clip(a_pct, eps, None)

    psi = np.sum((a_pct - e_pct) * np.log(a_pct / e_pct))
    return float(psi)

def effect_size_cohens_d(x0: np.ndarray, x1: np.ndarray) -> float:
    x0 = x0[np.isfinite(x0)]
    x1 = x1[np.isfinite(x1)]
    if x0.size < 2 or x1.size < 2:
        return np.nan
    m0, m1 = x0.mean(), x1.mean()
    s0, s1 = x0.std(ddof=1), x1.std(ddof=1)
    sp = np.sqrt(((x0.size - 1) * s0**2 + (x1.size - 1) * s1**2) / max((x0.size + x1.size - 2), 1))
    if sp == 0:
        return np.nan
    return float((m1 - m0) / sp)

def safe_auc(y_true: np.ndarray, scores: np.ndarray) -> float:
    # AUC requires both classes
    if len(np.unique(y_true)) < 2:
        return np.nan
    try:
        return float(roc_auc_score(y_true, scores))
    except Exception:
        return np.nan

def plot_feature_hist_overlay(train: pd.Series, other: pd.Series, title: str, other_label: str):
    plt.figure()
    # density hist overlay
    plt.hist(train.dropna().to_numpy(), bins=60, density=True, alpha=0.5, label="train")
    plt.hist(other.dropna().to_numpy(), bins=60, density=True, alpha=0.5, label=other_label)
    plt.title(title)
    plt.legend()

def plot_box_by_class(val_df: pd.DataFrame, feature: str):
    plt.figure()
    x0 = val_df.loc[val_df["Class"] == 0, feature].dropna().to_numpy()
    x1 = val_df.loc[val_df["Class"] == 1, feature].dropna().to_numpy()
    plt.boxplot([x0, x1], labels=["Class=0", "Class=1"], showfliers=False)
    plt.title(f"Boxplot by Class: {feature}")

def plot_scatter_pca(pca_train: np.ndarray, pca_val: np.ndarray, y_val: np.ndarray):
    plt.figure()
    plt.scatter(pca_train[:, 0], pca_train[:, 1], s=6, alpha=0.25, label="train")
    plt.scatter(pca_val[y_val == 0, 0], pca_val[y_val == 0, 1], s=10, alpha=0.7, label="val class=0")
    plt.scatter(pca_val[y_val == 1, 0], pca_val[y_val == 1, 1], s=25, alpha=0.9, label="val class=1")
    plt.title("PCA (fit=train) - Train vs Val (colored by Class)")
    plt.legend()

In [4]:
# =============================
# 1) Load data
# =============================
train_path = "open/train.csv"
val_path = "open/val.csv"
test_path = "open/test.csv"
sub_path = "open/sample_submission.csv"

train = pd.read_csv(train_path)
val = pd.read_csv(val_path)
test = pd.read_csv(test_path)
sub = pd.read_csv(sub_path)

print("train:", train.shape, "val:", val.shape, "test:", test.shape, "sub:", sub.shape)

# Columns
id_col = "ID"
class_col = "Class"
feature_cols = [c for c in train.columns if c != id_col]

# Ensure expected columns
assert id_col in train.columns and id_col in val.columns and id_col in test.columns, "ID column missing"
assert all(c in val.columns for c in feature_cols), "val missing some features"
assert all(c in test.columns for c in feature_cols), "test missing some features"
assert class_col in val.columns, "val missing Class"

# Ensure numeric
train = ensure_numeric(train, feature_cols)
val = ensure_numeric(val, feature_cols + [class_col])
test = ensure_numeric(test, feature_cols)

# Split
X_train = train[feature_cols].copy()
X_val = val[feature_cols].copy()
y_val = val[class_col].to_numpy().astype(int)
X_test = test[feature_cols].copy()

# Save a basic meta json
meta = {
    "train_shape": train.shape,
    "val_shape": val.shape,
    "test_shape": test.shape,
    "n_features": len(feature_cols),
    "feature_cols": feature_cols
}
with open(os.path.join(OUT_DIR, "meta.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)
print(f"[SAVE] {os.path.join(OUT_DIR, 'meta.json')}")

train: (113842, 31) val: (28462, 32) test: (142503, 31) sub: (142503, 2)
[SAVE] eda_outputs\meta.json


In [5]:
# =============================
# 2) Integrity / Quality EDA
# =============================
print("\n[EDA] 2) Integrity / Quality")

# ID uniqueness
def id_report(df: pd.DataFrame, name: str) -> Dict:
    n = df.shape[0]
    uniq = df[id_col].nunique()
    dup = n - uniq
    return {"dataset": name, "rows": n, "unique_ids": uniq, "duplicate_ids": dup}

id_rep = pd.DataFrame([
    id_report(train, "train"),
    id_report(val, "val"),
    id_report(test, "test"),
])
save_df(id_rep, "id_uniqueness")

# ID overlap across splits
train_ids = set(train[id_col].tolist())
val_ids = set(val[id_col].tolist())
test_ids = set(test[id_col].tolist())

overlap_rep = pd.DataFrame([
    {"pair": "train∩val", "overlap": len(train_ids & val_ids)},
    {"pair": "train∩test", "overlap": len(train_ids & test_ids)},
    {"pair": "val∩test", "overlap": len(val_ids & test_ids)},
])
save_df(overlap_rep, "id_overlap")

# NaN/Inf counts
def nan_inf_report(X: pd.DataFrame, name: str) -> pd.DataFrame:
    nan_cnt = X.isna().sum()
    inf_cnt = pd.Series(np.isinf(X.to_numpy()).sum(axis=0), index=X.columns)
    out = pd.DataFrame({
        "feature": X.columns,
        f"{name}_nan": nan_cnt.values,
        f"{name}_inf": inf_cnt.values
    })
    return out

nan_inf = nan_inf_report(X_train, "train") \
    .merge(nan_inf_report(X_val, "val"), on="feature") \
    .merge(nan_inf_report(X_test, "test"), on="feature")

save_df(nan_inf, "nan_inf_counts")

# Near-constant features (variance small)
var_train = X_train.var()
near_const = pd.DataFrame({
    "feature": var_train.index,
    "train_var": var_train.values
}).sort_values("train_var", ascending=True)

save_df(near_const, "train_variance_sorted")

# Quick plot of variance distribution
plt.figure()
plt.hist(near_const["train_var"].to_numpy(), bins=60)
plt.title("Train feature variance distribution")
save_fig("train_variance_hist")


[EDA] 2) Integrity / Quality
[SAVE] eda_outputs\id_uniqueness.csv (3 rows, 4 cols)
[SAVE] eda_outputs\id_overlap.csv (3 rows, 2 cols)
[SAVE] eda_outputs\nan_inf_counts.csv (30 rows, 7 cols)
[SAVE] eda_outputs\train_variance_sorted.csv (30 rows, 2 cols)
[SAVE] eda_outputs\train_variance_hist.png


In [6]:
# =============================
# 3) Univariate summary (train/val/test)
# =============================
print("\n[EDA] 3) Univariate Summary")

sum_train = summarize_basic(train, feature_cols, "train")
sum_val = summarize_basic(val, feature_cols, "val")
sum_test = summarize_basic(test, feature_cols, "test")

summary_all = sum_train.merge(sum_val, on="feature").merge(sum_test, on="feature")
save_df(summary_all, "summary_univariate_all")

# Tail heaviness ranking (by kurtosis, abs skew) on train
tail_rank = summary_all[["feature", "train_skew", "train_kurtosis", "train_q99", "train_max", "train_min"]] \
    .assign(abs_skew=lambda d: d["train_skew"].abs()) \
    .sort_values(["train_kurtosis", "abs_skew"], ascending=False)

save_df(tail_rank, "train_tail_rank")

# Plot example: top 6 heavy-tail features overlay train vs test
top6 = tail_rank["feature"].head(6).tolist()
for c in top6:
    plot_feature_hist_overlay(train[c], test[c], title=f"Train vs Test distribution: {c}", other_label="test")
    save_fig(f"hist_overlay_train_test_{c}")


[EDA] 3) Univariate Summary
[SAVE] eda_outputs\summary_univariate_all.csv (30 rows, 49 cols)
[SAVE] eda_outputs\train_tail_rank.csv (30 rows, 7 cols)
[SAVE] eda_outputs\hist_overlay_train_test_V28.png
[SAVE] eda_outputs\hist_overlay_train_test_V23.png
[SAVE] eda_outputs\hist_overlay_train_test_V29.png
[SAVE] eda_outputs\hist_overlay_train_test_V8.png
[SAVE] eda_outputs\hist_overlay_train_test_V21.png
[SAVE] eda_outputs\hist_overlay_train_test_V20.png


In [7]:
# =============================
# 4) Drift EDA (train vs val/test)
# =============================
print("\n[EDA] 4) Drift Metrics")

drift_rows = []
for c in feature_cols:
    a = X_train[c].to_numpy()
    b_val = X_val[c].to_numpy()
    b_test = X_test[c].to_numpy()

    # KS & Wasserstein (finite only)
    a_f = a[np.isfinite(a)]
    v_f = b_val[np.isfinite(b_val)]
    t_f = b_test[np.isfinite(b_test)]

    if a_f.size > 20 and v_f.size > 20:
        ks_v = ks_2samp(a_f, v_f).statistic
        wd_v = wasserstein_distance(a_f, v_f)
        psi_v = compute_psi(a_f, v_f, bins=20)
    else:
        ks_v = wd_v = psi_v = np.nan

    if a_f.size > 20 and t_f.size > 20:
        ks_t = ks_2samp(a_f, t_f).statistic
        wd_t = wasserstein_distance(a_f, t_f)
        psi_t = compute_psi(a_f, t_f, bins=20)
    else:
        ks_t = wd_t = psi_t = np.nan

    drift_rows.append({
        "feature": c,
        "KS_train_val": ks_v, "Wass_train_val": wd_v, "PSI_train_val": psi_v,
        "KS_train_test": ks_t, "Wass_train_test": wd_t, "PSI_train_test": psi_t
    })

drift = pd.DataFrame(drift_rows)

# Rank drift: primarily PSI_train_test then KS_train_test
drift_rank = drift.sort_values(["PSI_train_test", "KS_train_test"], ascending=False)
save_df(drift_rank, "drift_ranked")

# Plot top 6 drift features overlay
top6_drift = drift_rank["feature"].head(6).tolist()
for c in top6_drift:
    plot_feature_hist_overlay(train[c], test[c], title=f"DRIFT TOP - Train vs Test: {c}", other_label="test")
    save_fig(f"drift_top_hist_{c}")


[EDA] 4) Drift Metrics
[SAVE] eda_outputs\drift_ranked.csv (30 rows, 7 cols)
[SAVE] eda_outputs\drift_top_hist_V3.png
[SAVE] eda_outputs\drift_top_hist_V28.png
[SAVE] eda_outputs\drift_top_hist_V14.png
[SAVE] eda_outputs\drift_top_hist_V6.png
[SAVE] eda_outputs\drift_top_hist_V23.png
[SAVE] eda_outputs\drift_top_hist_V17.png


In [8]:
# =============================
# 5) Label-based EDA on Val (fraud signals)
# =============================
print("\n[EDA] 5) Val Label-based Analysis")

val_counts = pd.DataFrame([
    {"class": 0, "count": int((y_val == 0).sum())},
    {"class": 1, "count": int((y_val == 1).sum())},
])
val_counts["ratio"] = val_counts["count"] / val_counts["count"].sum()
save_df(val_counts, "val_class_distribution")

# Feature separation stats: ranksum p-value, Cohen's d, median diff, AUC (single feature)
sep_rows = []
for c in feature_cols:
    x0 = X_val.loc[val[class_col] == 0, c].to_numpy()
    x1 = X_val.loc[val[class_col] == 1, c].to_numpy()

    x0f = x0[np.isfinite(x0)]
    x1f = x1[np.isfinite(x1)]

    if x0f.size > 5 and x1f.size > 2:
        p = ranksums(x1f, x0f).pvalue  # (fraud vs normal)
        d = effect_size_cohens_d(x0f, x1f)
        med_diff = float(np.median(x1f) - np.median(x0f))
        # AUC uses raw feature as score
        auc = safe_auc(y_val, X_val[c].to_numpy())
        # "fraud outside normal IQR?" indicators
        lb, ub = iqr_bounds(pd.Series(x0f))
        fraud_med = float(np.median(x1f))
        outside_iqr = int(not (lb < fraud_med < ub))
    else:
        p = d = med_diff = auc = np.nan
        outside_iqr = 0

    sep_rows.append({
        "feature": c,
        "ranksum_p": p,
        "cohens_d(fraud-normal)": d,
        "median_diff(fraud-normal)": med_diff,
        "auc_single_feature": auc,
        "fraud_median_outside_normal_IQR": outside_iqr,
        "val_normal_n": int(x0f.size),
        "val_fraud_n": int(x1f.size),
    })

sep = pd.DataFrame(sep_rows)

# rank: smallest p (strong diff), then |d| (effect size)
sep["abs_d"] = sep["cohens_d(fraud-normal)"].abs()
sep_rank = sep.sort_values(["ranksum_p", "abs_d"], ascending=[True, False])
save_df(sep_rank, "val_feature_separation_ranked")

# Plot top 8 separated features: boxplot by class + overlay hist
top8_sep = sep_rank["feature"].head(8).tolist()
for c in top8_sep:
    plot_box_by_class(val, c)
    save_fig(f"box_by_class_{c}")

    plt.figure()
    x0 = X_val.loc[val[class_col] == 0, c].dropna().to_numpy()
    x1 = X_val.loc[val[class_col] == 1, c].dropna().to_numpy()
    plt.hist(x0, bins=60, density=True, alpha=0.55, label="Class=0")
    plt.hist(x1, bins=60, density=True, alpha=0.55, label="Class=1")
    plt.title(f"Val distribution by Class: {c}")
    plt.legend()
    save_fig(f"val_hist_by_class_{c}")


[EDA] 5) Val Label-based Analysis
[SAVE] eda_outputs\val_class_distribution.csv (2 rows, 3 cols)
[SAVE] eda_outputs\val_feature_separation_ranked.csv (30 rows, 9 cols)
[SAVE] eda_outputs\box_by_class_V10.png


C:\Users\xrtwin\AppData\Local\Temp\ipykernel_23116\3769959870.py:121: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot([x0, x1], labels=["Class=0", "Class=1"], showfliers=False)


[SAVE] eda_outputs\val_hist_by_class_V10.png
[SAVE] eda_outputs\box_by_class_V14.png


C:\Users\xrtwin\AppData\Local\Temp\ipykernel_23116\3769959870.py:121: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot([x0, x1], labels=["Class=0", "Class=1"], showfliers=False)


[SAVE] eda_outputs\val_hist_by_class_V14.png
[SAVE] eda_outputs\box_by_class_V11.png


...

[SAVE] eda_outputs\val_hist_by_class_V3.png
[SAVE] eda_outputs\box_by_class_V9.png


C:\Users\xrtwin\AppData\Local\Temp\ipykernel_23116\3769959870.py:121: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot([x0, x1], labels=["Class=0", "Class=1"], showfliers=False)


[SAVE] eda_outputs\val_hist_by_class_V9.png
[SAVE] eda_outputs\box_by_class_V2.png


C:\Users\xrtwin\AppData\Local\Temp\ipykernel_23116\3769959870.py:121: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot([x0, x1], labels=["Class=0", "Class=1"], showfliers=False)


[SAVE] eda_outputs\val_hist_by_class_V2.png


In [9]:
# =============================
# 6) Correlation / Redundancy EDA (train)
# =============================
print("\n[EDA] 6) Correlation & Redundancy")

corr = X_train.corr(method="pearson")
# Save top correlated pairs
corr_abs = corr.abs()
np.fill_diagonal(corr_abs.values, 0.0)

# extract top pairs
pairs = []
cols = corr_abs.columns.tolist()
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        pairs.append((cols[i], cols[j], corr.iloc[i, j], corr_abs.iloc[i, j]))
top_corr = pd.DataFrame(pairs, columns=["f1", "f2", "corr", "abs_corr"]).sort_values("abs_corr", ascending=False)
save_df(top_corr.head(200), "top_corr_pairs_train")

# Heatmap-like visualization (matplotlib imshow)
plt.figure(figsize=(8, 7))
plt.imshow(corr.values, aspect="auto")
plt.colorbar()
plt.title("Train Pearson Correlation (V1..V30)")
plt.xticks(range(len(feature_cols)), feature_cols, rotation=90, fontsize=6)
plt.yticks(range(len(feature_cols)), feature_cols, fontsize=6)
save_fig("train_corr_heatmap")


[EDA] 6) Correlation & Redundancy
[SAVE] eda_outputs\top_corr_pairs_train.csv (200 rows, 4 cols)
[SAVE] eda_outputs\train_corr_heatmap.png


In [10]:
# =============================
# 7) PCA Visualization (fit=train only)
# =============================
print("\n[EDA] 7) PCA Visualization (fit=train)")

pca = PCA(n_components=2, random_state=RANDOM_SEED)
pca_train = pca.fit_transform(X_train.fillna(0).to_numpy())  # fit on train only
pca_val = pca.transform(X_val.fillna(0).to_numpy())
pca_test = pca.transform(X_test.fillna(0).to_numpy())

plot_scatter_pca(pca_train, pca_val, y_val)
save_fig("pca_train_val_scatter")

# Train vs Test PCA scatter (no labels)
plt.figure()
plt.scatter(pca_train[:, 0], pca_train[:, 1], s=6, alpha=0.25, label="train")
plt.scatter(pca_test[:, 0], pca_test[:, 1], s=6, alpha=0.25, label="test")
plt.title("PCA (fit=train) - Train vs Test")
plt.legend()
save_fig("pca_train_test_scatter")

# Save PCA explained variance
pca_info = pd.DataFrame([{
    "pca_explained_var_ratio_dim1": float(pca.explained_variance_ratio_[0]),
    "pca_explained_var_ratio_dim2": float(pca.explained_variance_ratio_[1]),
}])
save_df(pca_info, "pca_explained_variance")


[EDA] 7) PCA Visualization (fit=train)
[SAVE] eda_outputs\pca_train_val_scatter.png
[SAVE] eda_outputs\pca_train_test_scatter.png
[SAVE] eda_outputs\pca_explained_variance.csv (1 rows, 2 cols)


In [11]:
# =============================
# 8) Anomaly-score EDA with IsolationForest (baseline-style)
# =============================
print("\n[EDA] 8) IsolationForest Score Analysis (fit=train)")

# initial contamination estimate from val label ratio (for reference)
val_fraud_ratio = float((y_val == 1).mean())
ref = pd.DataFrame([{"val_fraud_ratio": val_fraud_ratio}])
save_df(ref, "val_fraud_ratio_reference")

def if_predict_01(pred_1_minus1: np.ndarray) -> np.ndarray:
    out = pred_1_minus1.copy()
    out[out == 1] = 0
    out[out == -1] = 1
    return out

# Fit IF on train only
base_contam = max(min(val_fraud_ratio, 0.02), 0.0001)  # clamp for safety
iso = IsolationForest(
    n_estimators=1000,
    contamination=base_contam,
    random_state=RANDOM_SEED,
    n_jobs=-1
)
iso.fit(X_train.fillna(0).to_numpy())

# scores: decision_function (higher=more normal), score_samples (higher=more normal in sklearn)
val_pred = if_predict_01(iso.predict(X_val.fillna(0).to_numpy()))
macro = f1_score(y_val, val_pred, average="macro")
cm = confusion_matrix(y_val, val_pred)

with open(os.path.join(OUT_DIR, "if_baseline_val_report.txt"), "w", encoding="utf-8") as f:
    f.write(f"contamination={base_contam}\n")
    f.write(f"macro_f1={macro}\n\n")
    f.write("confusion_matrix:\n")
    f.write(str(cm) + "\n\n")
    f.write("classification_report:\n")
    f.write(classification_report(y_val, val_pred))
print(f"[SAVE] {os.path.join(OUT_DIR, 'if_baseline_val_report.txt')}")

# score distributions
val_scores = iso.decision_function(X_val.fillna(0).to_numpy())  # higher = more normal
plt.figure()
plt.hist(val_scores[y_val == 0], bins=60, density=True, alpha=0.55, label="val class=0")
plt.hist(val_scores[y_val == 1], bins=60, density=True, alpha=0.55, label="val class=1")
plt.title("IsolationForest decision_function on Val (fit=train)")
plt.legend()
save_fig("if_val_score_hist")

# top-k recall check (how many frauds in most anomalous k)
# most anomalous => lowest decision_function
order = np.argsort(val_scores)  # ascending => most anomalous first
ks = [10, 20, 30, 50, 100, 200, 300, 500, 1000]
rec_rows = []
fraud_total = int((y_val == 1).sum())
for k in ks:
    k = min(k, len(y_val))
    picked = y_val[order[:k]]
    hit = int((picked == 1).sum())
    rec_rows.append({
        "k": k,
        "hits_fraud_in_topk": hit,
        "fraud_total": fraud_total,
        "recall_at_k": hit / max(fraud_total, 1)
    })
rec_df = pd.DataFrame(rec_rows)
save_df(rec_df, "if_recall_at_k")

# contamination sweep (macro F1 curve)
contams = np.unique(np.concatenate([
    np.array([0.0005, 0.0008, 0.001, 0.0012, 0.0015, 0.002, 0.003, 0.005]),
    np.linspace(0.0005, 0.005, 10)
]))
sweep_rows = []
Xtr = X_train.fillna(0).to_numpy()
Xva = X_val.fillna(0).to_numpy()
for c in contams:
    model = IsolationForest(n_estimators=600, contamination=float(c), random_state=RANDOM_SEED, n_jobs=-1)
    model.fit(Xtr)  # fit=train only
    pred = if_predict_01(model.predict(Xva))
    macro_f1 = f1_score(y_val, pred, average="macro")
    # count predicted fraud
    n_pred1 = int((pred == 1).sum())
    sweep_rows.append({"contamination": float(c), "macro_f1": float(macro_f1), "predicted_ones": n_pred1})

sweep = pd.DataFrame(sweep_rows).sort_values("contamination")
save_df(sweep, "if_contamination_sweep")

plt.figure()
plt.plot(sweep["contamination"], sweep["macro_f1"], marker="o")
plt.title("Macro F1 vs contamination (IF, fit=train)")
plt.xlabel("contamination")
plt.ylabel("macro F1")
save_fig("if_contam_sweep_macro_f1")

plt.figure()
plt.plot(sweep["contamination"], sweep["predicted_ones"], marker="o")
plt.title("Predicted #ones vs contamination (IF, fit=train)")
plt.xlabel("contamination")
plt.ylabel("# predicted Class=1")
save_fig("if_contam_sweep_pred_ones")


[EDA] 8) IsolationForest Score Analysis (fit=train)
[SAVE] eda_outputs\val_fraud_ratio_reference.csv (1 rows, 1 cols)
[SAVE] eda_outputs\if_baseline_val_report.txt
[SAVE] eda_outputs\if_val_score_hist.png
[SAVE] eda_outputs\if_recall_at_k.csv (9 rows, 4 cols)
[SAVE] eda_outputs\if_contamination_sweep.csv (12 rows, 3 cols)
[SAVE] eda_outputs\if_contam_sweep_macro_f1.png
[SAVE] eda_outputs\if_contam_sweep_pred_ones.png


In [12]:
# =============================
# 9) Sample-level FP/FN Case Study (Val)
# =============================
print("\n[EDA] 9) Sample-level FP/FN analysis (Val)")

val_df_case = val[[id_col] + feature_cols + [class_col]].copy()
val_df_case["if_score"] = val_scores
val_df_case["pred"] = val_pred

fp = val_df_case[(val_df_case[class_col] == 0) & (val_df_case["pred"] == 1)].copy()
fn = val_df_case[(val_df_case[class_col] == 1) & (val_df_case["pred"] == 0)].copy()
tp = val_df_case[(val_df_case[class_col] == 1) & (val_df_case["pred"] == 1)].copy()

# Most anomalous among FP (lowest score)
fp_top = fp.sort_values("if_score").head(50)
fn_top = fn.sort_values("if_score").head(50)  # suspicious but missed
tp_top = tp.sort_values("if_score").head(50)

save_df(fp_top[[id_col, class_col, "pred", "if_score"] + feature_cols], "val_fp_top50_by_anomaly_score")
save_df(fn_top[[id_col, class_col, "pred", "if_score"] + feature_cols], "val_fn_top50_by_anomaly_score")
save_df(tp_top[[id_col, class_col, "pred", "if_score"] + feature_cols], "val_tp_top50_by_anomaly_score")

# Which features are extreme in FP compared to normal distribution?
# Compute z-like using train mean/std (fit=train statistics only)
mu = X_train.mean()
sd = X_train.std().replace(0, np.nan)

def extreme_feature_profile(samples: pd.DataFrame, name: str, topn: int = 12):
    Xs = samples[feature_cols]
    z = (Xs - mu) / sd
    z_abs_mean = z.abs().mean().sort_values(ascending=False)
    out = pd.DataFrame({"feature": z_abs_mean.index, f"{name}_mean_abs_z": z_abs_mean.values})
    save_df(out.head(30), f"{name}_extreme_features_by_mean_abs_z")
    # plot topn
    top_features = z_abs_mean.head(topn).index.tolist()
    plt.figure(figsize=(9, 4))
    plt.bar(range(topn), z_abs_mean.head(topn).to_numpy())
    plt.xticks(range(topn), top_features, rotation=45, ha="right")
    plt.title(f"{name}: Top extreme features (mean |z|, stats from train)")
    save_fig(f"{name}_extreme_features_bar")

if len(fp) > 0:
    extreme_feature_profile(fp_top, "FP_top50")
if len(fn) > 0:
    extreme_feature_profile(fn_top, "FN_top50")
if len(tp) > 0:
    extreme_feature_profile(tp_top, "TP_top50")


[EDA] 9) Sample-level FP/FN analysis (Val)
[SAVE] eda_outputs\val_fp_top50_by_anomaly_score.csv (15 rows, 34 cols)
[SAVE] eda_outputs\val_fn_top50_by_anomaly_score.csv (19 rows, 34 cols)
[SAVE] eda_outputs\val_tp_top50_by_anomaly_score.csv (11 rows, 34 cols)
[SAVE] eda_outputs\FP_top50_extreme_features_by_mean_abs_z.csv (30 rows, 2 cols)
[SAVE] eda_outputs\FP_top50_extreme_features_bar.png
[SAVE] eda_outputs\FN_top50_extreme_features_by_mean_abs_z.csv (30 rows, 2 cols)
[SAVE] eda_outputs\FN_top50_extreme_features_bar.png
[SAVE] eda_outputs\TP_top50_extreme_features_by_mean_abs_z.csv (30 rows, 2 cols)
[SAVE] eda_outputs\TP_top50_extreme_features_bar.png


In [13]:
# =============================
# 10) Optional: Nonlinear embedding (PaCMAP/UMAP) if installed
# =============================
print("\n[EDA] 10) Optional nonlinear embedding (if available)")

def try_pacmap():
    try:
        import pacmap
        # Fit on train only, transform val
        Xtr = X_train.fillna(0).to_numpy()
        Xva = X_val.fillna(0).to_numpy()
        emb = pacmap.PaCMAP(n_components=2, random_state=RANDOM_SEED, num_iters=400, verbose=False)
        tr2 = emb.fit_transform(Xtr, init="pca")
        va2 = emb.transform(Xva, basis=Xtr)
        plt.figure()
        plt.scatter(tr2[:, 0], tr2[:, 1], s=6, alpha=0.25, label="train")
        plt.scatter(va2[y_val == 0, 0], va2[y_val == 0, 1], s=10, alpha=0.7, label="val class=0")
        plt.scatter(va2[y_val == 1, 0], va2[y_val == 1, 1], s=25, alpha=0.9, label="val class=1")
        plt.title("PaCMAP (fit=train) - Train vs Val (colored by Class)")
        plt.legend()
        save_fig("pacmap_train_val_scatter")
        return True
    except Exception as e:
        print("[SKIP] PaCMAP not available or failed:", repr(e))
        return False

def try_umap():
    try:
        import umap
        Xtr = X_train.fillna(0).to_numpy()
        Xva = X_val.fillna(0).to_numpy()
        reducer = umap.UMAP(n_components=2, random_state=RANDOM_SEED)
        tr2 = reducer.fit_transform(Xtr)      # fit=train only
        va2 = reducer.transform(Xva)
        plt.figure()
        plt.scatter(tr2[:, 0], tr2[:, 1], s=6, alpha=0.25, label="train")
        plt.scatter(va2[y_val == 0, 0], va2[y_val == 0, 1], s=10, alpha=0.7, label="val class=0")
        plt.scatter(va2[y_val == 1, 0], va2[y_val == 1, 1], s=25, alpha=0.9, label="val class=1")
        plt.title("UMAP (fit=train) - Train vs Val (colored by Class)")
        plt.legend()
        save_fig("umap_train_val_scatter")
        return True
    except Exception as e:
        print("[SKIP] UMAP not available or failed:", repr(e))
        return False

_ = try_pacmap()
_ = try_umap()


[EDA] 10) Optional nonlinear embedding (if available)


[SAVE] eda_outputs\pacmap_train_val_scatter.png
[SKIP] UMAP not available or failed: ModuleNotFoundError("No module named 'umap'")


In [14]:
# =============================
# 11) Final: Executive summary text
# =============================
print("\n[EDA] 11) Write executive summary")

# top drift features
top_drift = drift_rank.head(10)[["feature", "PSI_train_test", "KS_train_test", "Wass_train_test"]]
top_sep = sep_rank.head(10)[["feature", "ranksum_p", "abs_d", "auc_single_feature", "fraud_median_outside_normal_IQR"]]

summary_lines = []
summary_lines.append("=== Executive Summary ===")
summary_lines.append(f"- Train rows: {len(train)}, Val rows: {len(val)}, Test rows: {len(test)}")
summary_lines.append(f"- #features: {len(feature_cols)}")
summary_lines.append(f"- Val fraud ratio: {val_fraud_ratio:.6f} (used as reference for contamination)")
summary_lines.append("")
summary_lines.append("[Top-10 Drift features: train vs test]")
summary_lines.append(top_drift.to_string(index=False))
summary_lines.append("")
summary_lines.append("[Top-10 Separation features on Val (label-based)]")
summary_lines.append(top_sep.to_string(index=False))
summary_lines.append("")
summary_lines.append("Next actions:")
summary_lines.append("- If drift is high, consider excluding those features or applying robust transforms (fit on train only).")
summary_lines.append("- Use separation ranking as 'hint' for feature selection, but avoid fitting transforms on val.")
summary_lines.append("- Analyze FP/FN case tables to identify features causing false positives and consider post-filtering rules.")

summary_path = os.path.join(OUT_DIR, "executive_summary.txt")
with open(summary_path, "w", encoding="utf-8") as f:
    f.write("\n".join(summary_lines))
print(f"[SAVE] {summary_path}")

print("\nDONE. Check ./eda_outputs/ for CSV/PNG/TXT outputs.")


[EDA] 11) Write executive summary
[SAVE] eda_outputs\executive_summary.txt

DONE. Check ./eda_outputs/ for CSV/PNG/TXT outputs.
